# Problem Statement

We would like to add a feature for recommended tickers and weights. 

## High Level Design

* This can be run weekly/fortnightly. 

* We will download data for the top 50 tickers for a given index. 

* Get the sharpe ratios based on criteria - 5 years data, 2 years data, 1 year data. Average them out and take the top 20, discard the rest

* Use the top 20, run Efficient Frontier for min volatility and max sharpe. Display these data with some text explanation (**ChatGPT??**)
